In [1]:
import os
import json
import rapidfuzz as rf

In [2]:
from collections import defaultdict

In [3]:
import random
import json

In [4]:
def load_files(folder_path):
    content_list = []
    meta_list = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):  # Ensure it's a JSON file
            file_path = os.path.join(folder_path, filename)
            
            with open(file_path, "r", encoding="utf-8") as file:
                try:
                    data = json.load(file)
                    if "content" in data:
                        content_list.append(data["content"])
                        meta_list.append(data["meta"])
                except json.JSONDecodeError as e:
                    print(f"Error loading {filename}: {e}")
    print("Loaded", len(content_list), "items from JSON files.")
    return content_list, meta_list

In [5]:
content_list_crime, meta_list_crime = load_files(".\data\scraped\kzp_in_ko")

Loaded 3020 items from JSON files.


In [6]:
content_list_civil, meta_list_civil = load_files(".\data\scraped\cpp_in__co_ali_go__in_vs")

Loaded 16312 items from JSON files.


In [25]:
duplicates_count = 0
lengths = []
lengths_duplicates = []
for content in content_list_civil:
    if "jedro" not in content:
        continue
    if "izrek" not in content:
        continue
    if content["jedro"] == content["izrek"]:
        duplicates_count +=1
        lengths_duplicates.append(len(content["obrazložitev"]))
    lengths.append(len(content["obrazložitev"]))

In [20]:
duplicates_count, len(content_list_civil)

(1509, 16312)

In [26]:
sum(lengths) / len(lengths)

5192.850409964323

In [27]:
sum(lengths_duplicates) / len(lengths_duplicates)

2820.85155732273

In [42]:
def calculate_verbatim(content_list, meta_list, is_verbatim=True):
    dataset_crime = []
    verbatim_count = 0
    categories_processed = set()
    processed_count = 0
    for content, meta in zip(content_list, meta_list):
        if "jedro" not in content:
            continue
        if "izrek" not in content:
            continue
        if content["jedro"] == content["izrek"]:
            continue
        processed_count += 1
        jedro = content["jedro"]
        celota = content["obrazložitev"]
    
        found = 0
        verbatim = []
        sentences = [s.strip() for s in jedro.split('.') if s.strip()]
        for sentence in sentences: 
            match = rf.fuzz.partial_ratio_alignment(sentence, celota)
            if match.score > 80:
                found += 1
                verbatim.append(celota[match.dest_start: match.dest_end])
        if (found >= len(sentences) * 0.9) == is_verbatim:
            verbatim_count += 1
    
            if meta["Področje"] not in categories_processed:
                dataset_crime.append({"obrazložitev": celota, "jedro": jedro, "odseki": verbatim})
                categories_processed.add(meta["Področje"])
    print(verbatim_count, len(content_list))
    print(len(dataset_crime), len(categories_processed))
    print(processed_count)
    return dataset_crime


In [43]:
dataset_civil = calculate_verbatim(content_list_civil, meta_list_civil)

9199 16312
601 601
14468


In [44]:
dataset_crime = calculate_verbatim(content_list_crime, meta_list_crime)

2127 3020
32 32
3014


In [45]:
dataset_civil_non_verbatim = calculate_verbatim(content_list_civil, meta_list_civil, is_verbatim=False)

5269 16312
511 511
14468


In [46]:
dataset_crime_non_verbatim = calculate_verbatim(content_list_crime, meta_list_crime, is_verbatim=False)

887 3020
28 28
3014


In [48]:
9199 + 5269 + 1509

15977

In [41]:
16312 - 15977

335

In [36]:
6599 + 9378 + 1509

17486

In [66]:
2127/ 3020

0.7043046357615894

In [55]:
9595/ 16312

0.5882172633643943

In [49]:
dataset_civil[1]["jedro"]

'Revizija je dopuščena glede vprašanj:\n\n- ali je bilo spoštovano načelo sorazmernosti med potrebami kazenskega postopka in lastninsko pravico tožnika, ki se kaže skozi trajanje zasega vozila in\n\n- ali je konkretna škoda izkazana tudi v višini zmanjšane vrednosti vozila zaradi časa, ki je potekel med odvzemom in vrnitvijo vozila.'

In [50]:
dataset_crime[1]["jedro"]

'Zvočno snemanje obsojenca in posredovanje posnetka organom pregona zaradi kazenskega pregona obsojenca, nedvomno pomeni poseg v obsojenčevo pravico do zasebnosti iz 35. člena Ustave. Vendar pa je bil po presoji Vrhovnega sodišča, ob uporabi načela sorazmernosti, poseg v obsojenčevo pravico do zasebnosti v konkretnem primeru dopusten. Zvočni posnetek, ki ga je sodišče upoštevalo kot dokaz, je namreč nastal v trenutku obsojenčevega izvrševanja izredno hudega kaznivega dejanja - napeljevanja h kaznivemu dejanju umora po 2. točki drugega odstavka v zvezi s prvim odstavkom 127. člena KZ. Pravica do osebne varnosti oziroma življenja oškodovanke ima v konkretni situaciji nedvomno prednost pred osebnostno pravico obsojenca do glasu oziroma zasebnosti, v katero je bilo poseženo z zvočnim snemanjem v trenutku izvrševanja kaznivega dejanja in z izvedbo dokaza s poslušanjem posnetka na glavni obravnavi.'

In [51]:
sample1 = random.sample(dataset_civil, 30)
sample2 = random.sample(dataset_crime, 30)

sample3 = random.sample(dataset_civil_non_verbatim, 30)
sample4 = random.sample(dataset_crime_non_verbatim, 28)

In [53]:
with open("sample_test_verbatim.json", "w") as file:
    json.dump(sample1[:15] + sample2[:15], file, indent=4)

with open("sample_train_verbatim.json", "w") as file:
    json.dump(sample1[15:] + sample2[15:], file, indent=4)

with open("sample_test_non_verbatim.json", "w") as file:
    json.dump(sample3[:15] + sample4[:15], file, indent=4)

with open("sample_train_non_verbatim.json", "w") as file:
    json.dump(sample3[15:] + sample4[15:], file, indent=4)

In [45]:
# dataset_crime[1]["odseki"]

In [44]:
# dataset_crime[1]["obrazložitev"]

In [51]:
area_count = defaultdict(int)
for x in meta_list_civil:
    area_count[x["Področje"]] += 1

In [52]:
area_count

defaultdict(int,
            {'CIVILNO PROCESNO PRAVO': 7589,
             'CIVILNO PROCESNO PRAVO - OBLIGACIJSKO PRAVO': 746,
             'CIVILNO PROCESNO PRAVO - KAZENSKO PROCESNO PRAVO - ODŠKODNINSKO PRAVO': 2,
             'CIVILNO PROCESNO PRAVO - ZAVAROVALNO PRAVO': 105,
             'CIVILNO PROCESNO PRAVO - ODŠKODNINSKO PRAVO': 1062,
             'CIVILNO PROCESNO PRAVO - DRUŽINSKO PRAVO': 335,
             'CIVILNO PROCESNO PRAVO - OBLIGACIJSKO PRAVO - ODŠKODNINSKO PRAVO': 132,
             'CIVILNO PROCESNO PRAVO - ODŠKODNINSKO PRAVO - USTAVNO PRAVO': 41,
             'CIVILNO PROCESNO PRAVO - OBLIGACIJSKO PRAVO - STVARNO PRAVO': 63,
             'CIVILNO PROCESNO PRAVO - USTAVNO PRAVO': 57,
             'CIVILNO PROCESNO PRAVO - SODNE TAKSE': 53,
             'CIVILNO PROCESNO PRAVO - PRAVO INTELEKTUALNE LASTNINE': 62,
             'CIVILNO PROCESNO PRAVO - OBLIGACIJSKO PRAVO - POGODBENO PRAVO': 159,
             'CIVILNO PROCESNO PRAVO - NEPRAVDNO PRAVO - TELEKOMUNIKACIJE